# Co-Occurrence und KWIC

Wie du im letzten Teil gesehen hast, konnten wir relativ einfach Wordclouds generieren und anzeigen. Diese zeigen dir, welche Themen im Heise Newsticker behandelt werden.

Allerdings fehlen dir zur genaueren Interpretation noch *Kontextinformationen*. Die wirst du nun ermitteln

## Daten einladen

Du beginnst zunächst wieder mit dem Laden der linugistisch analysierten Daten aus der Datenbank. Das wird auch in Zukunft häufig dein erster Arbeitsschritt sein. Wenn du möchtest, kannst du auch nur eine Teilmenge mit `LIMIT` auswählen:

In [1]:
!pip install textacy

In [2]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [3]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles ORDER BY datePublished", sql, index_col="id")

## Vokabular bestimmen

Im ersten Schritt bestimmst du nun das Vokabular, das du analysieren möchtest. Dazu benötigst du wieder die Funktion, die die Felder in die Tokens zerlegt:

In [4]:
import regex as re
def single_words(df, field):
    return [w for words in df[field] for w in re.split(r'\||\#', words)]

Häufig ist das `nav`-Feld, das Substantive, Adjektive und Verben enthält, besonders gut für die Analyse geeignet. Mithilfe eines `Counter` bestimmst du die häufigsten Wörter in dem Feld, lässt aber die Stopwords weg. Die 10.000 häufigsten Wörter speicherst du in einer separaten Variable `voc` ab:

In [5]:
from collections import Counter
from spacy.lang.de.stop_words import STOP_WORDS

nav = Counter(single_words(df, "nav"))
for w in STOP_WORDS:
    nav[w] = 0
    
voc = [w[0] for w in nav.most_common(10000)]

In einem Fenster von fünf Wörtern werden nun die sog. *Co-Occurrences* ermittelt. Das sind die Wörter, die dort gemeinsam auftreten. Auch hier werden die Stopwords ausgelassen. Das Fenster wird vom "mittleren" Wort halb nach vorne und halb nach hinten gerechnet. Die Konstruktion mit dem `defaultdict` und dem `Counter` erspart dir viele `if`-Abfragen:

In [6]:
from collections import Counter, defaultdict
from tqdm import tqdm
import regex as re

# defaultdict liefert das Argument zurück, wenn key noch unbekannt
# in diesem Fall also einen neuen (leeren) Counter
c = defaultdict(lambda: Counter())

window = 5 # sollte ungerade sein
skip = (window - 1) // 2
for doc in tqdm(df["nav"]):
    # Stopwords eliminieren
    tokens = [w for w in re.split(r'\||\#', doc) if w not in STOP_WORDS]
    for i, w in enumerate(tokens):
        if w in voc:
            for j in range(max(0, i-skip), i):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1
            for j in range(i+1, min(i+1+skip, len(tokens))):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1

100%|██████████| 10409/10409 [04:52<00:00, 35.60it/s]


Nun kannst du ermitteln, welche Begriffe besonders häufig zusammen mit `Apple` vorkommen:

In [7]:
c["Apple"].most_common()

[('Google', 343),
 ('Watch', 280),
 ('Apple', 212),
 ('iPhone', 165),
 ('Jahr', 126),
 ('neu', 122),
 ('geben', 116),
 ('TV', 113),
 ('App', 112),
 ('Pay', 106),
 ('Epic', 100),
 ('iOS', 89),
 ('Silicon', 88),
 ('Facebook', 80),
 ('TV+', 79),
 ('Amazon', 77),
 ('Music', 77),
 ('Store', 76),
 ('bislang', 74),
 ('Series', 69),
 ('betonen', 66),
 ('Apps', 62),
 ('Entwickler', 61),
 ('macOS', 60),
 ('Prozent', 59),
 ('zudem', 58),
 ('iPad', 58),
 ('offenbaren', 57),
 ('Konzern', 56),
 ('setzen', 55),
 ('Game', 54),
 ('nutzen', 53),
 ('planen', 53),
 ('bieten', 48),
 ('Apples', 48),
 ('Auch', 47),
 ('Mac', 47),
 ('Gerät', 46),
 ('Microsoft', 46),
 ('mitteilen', 46),
 ('iPhones', 46),
 ('Nutzer', 45),
 ('sehen', 45),
 ('erstmals', 43),
 ('stellen', 43),
 ('führen', 43),
 ('nennen', 42),
 ('bleiben', 42),
 ('Funktion', 41),
 ('lassen', 41),
 ('integrieren', 41),
 ('verwenden', 41),
 ('bringen', 40),
 ('zeigen', 39),
 ('arbeiten', 39),
 ('unterstützen', 39),
 ('scheinen', 39),
 ('schreiben', 3

Das sieht schon sehr gut aus. Teste es noch mit `Google`:

In [8]:
c["Google"].most_common()

[('Apple', 343),
 ('Facebook', 251),
 ('Google', 148),
 ('Amazon', 134),
 ('Microsoft', 129),
 ('Maps', 117),
 ('Play', 114),
 ('Jahr', 94),
 ('geben', 87),
 ('Cloud', 86),
 ('Chrome', 76),
 ('Android', 64),
 ('Pay', 63),
 ('Datum', 62),
 ('Store', 61),
 ('Stadia', 61),
 ('Twitter', 58),
 ('ander', 57),
 ('neu', 56),
 ('Nutzer', 55),
 ('Funktion', 54),
 ('schreiben', 51),
 ('Unternehmen', 50),
 ('zeigen', 47),
 ('bieten', 46),
 ('Dienst', 45),
 ('arbeiten', 42),
 ('Blogbeitrag', 42),
 ('App', 41),
 ('Auch', 40),
 ('nutzen', 40),
 ('bekommen', 39),
 ('Co.', 36),
 ('bringen', 36),
 ('stellen', 36),
 ('Version', 35),
 ('setzen', 33),
 ('YouTube', 32),
 ('unterstützen', 32),
 ('Apps', 32),
 ('erklären', 31),
 ('sehen', 31),
 ('News', 31),
 ('führen', 29),
 ('Assistant', 29),
 ('finden', 29),
 ('lassen', 28),
 ('veröffentlichen', 28),
 ('laufen', 28),
 ('Prozent', 28),
 ('integrieren', 27),
 ('entwickeln', 27),
 ('lässt', 26),
 ('Gerät', 25),
 ('Klage', 25),
 ('ebenfalls', 25),
 ('verbesser

Auch dieses Ergebnis ist richtig gut und wie erwartet. Die Google-Dienste erscheinen eigentlich alle und du kannst damit feststellen, dass die Meldungen nicht nur die richtigen Wörter enthalten, sondern auch die richtigen Themen besprechen.

## Keywords in Context

Jetzt weißt du aber noch nicht, wie die Wörter wirklich verwendet werden. Dazu kann Keyword-in-Context [KWIC](https://de.wikipedia.org/wiki/Permutiertes_Register) helfen. Dazu werden für die Einzelmeldungen jeweils immer ein bestimmtes Festner von Buchstaben angezeigt, die um das gewählte Wort vorkommen. Die Funktion ist einfach implementiert, ein *regulärer Ausdruck* wird dazu dynamisch konstruiert:

In [9]:
import regex as re
def kwic(word, texts, window_size):
    res = []
    for text in texts:
        context = (window_size//2)*'.'
        kwic = context + r'\b' + word + r'\b' + context
        match = re.findall(kwic, text)
        for m in match:
            res.append(m)
            
    return res

Die Funktion erfordert ein Array von Strings, das kannst du aus dem `DataFrame` einfach ermitteln:

In [10]:
text = df["text"].map(str).values

Der Aufruf kann einen Augenblick dauern, weil es viele Meldungen mit `Apple` gibt:

In [11]:
kwic("Apple", text, 60)

['Ausgabe ("KI ohne Schnüffeln: Apple will zeigen, dass das geht").',
 '017. Das Verfahren erlaubt es Apple, verschiedene Kopien eines Sp',
 'eben Federated Learning nutzt Apple außerdem die sogenannte Diffe',
 'being" auf Android ausgebaut. Apple bietet für iOS unter anderem ',
 'te Größe auf der CES. Nachdem Apple große Erfolge damit feiern ko',
 'irekten Konkurrenten: Während Apple bei 1,6 Grad landet, erzielt ',
 'en AMD. Am Vergleich zwischen Apple und HTC wird allerdings ein N',
 'gramm Ware ausstößt. Nur weil Apple mehr Geld für seine Smartphon',
 'ter-Technik wie 2019, während Apple, MediaTek und Qualcomm auf di',
 'nd mobile Bezahlverfahren wie Apple Pay oder Google Pay auch in D',
 ' zwölf US-Plattformen Airbnb, Apple, Automattic, Cloudflare, Drop',
 ', Google sollte in 47.011 und Apple in 43.480 Fällen Auskunft geb',
 'em Netflix 2019 bereits durch Apple TV+ und Disney+ (startet in D',
 'ihnen zugesteht. Im Falle von Apple muss man jedenfalls feststell',
 'n Inhalten überaus

Jetzt kannst du die Einzelmeldungen sehen. Alle zu lesen, ist natürlich etwas aufwändiger, als die Co-Occurrence oben auszuwerten.

## *Context is King*

Um Texte genauer zu verstehen, sind Kontextinformationen eminent wichtig. Mit *Co-Occurrence* und *KWIC* hast du zwei Methoden kennengelernt, mit denen du den Kontext einfach bestimmen kannst.